In [ ]:
import time
import librosa
import soundfile as sf
from scipy.io import wavfile
import pandas as pd
import numpy as np
import csv

In [ ]:
wav_path = '/content/sample.wav'

In [ ]:
def read_librosa(path):
    start = time.perf_counter()
    y, sr = librosa.load(path, sr=48000, mono=True)
    duration = time.perf_counter() - start
    return y, sr, duration

def read_scipy(path):
    start = time.perf_counter()
    sr, y = wavfile.read(path)
    y = y.astype('float32') / np.max(np.abs(y))  # Normalize
    duration = time.perf_counter() - start
    return y, sr, duration

def read_soundfile(path):
    start = time.perf_counter()
    y, sr = sf.read(path)
    duration = time.perf_counter() - start
    return y, sr, duration

In [ ]:
mock_data = [
    ['0.0', '3.0', 'Cardinalis_cardinalis', 'Northern Cardinal', 0.95],
    ['3.0', '6.0', 'Corvus_brachyrhynchos', 'American Crow', 0.87],
]

def write_csv_standard(path):
    start = time.perf_counter()
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(['Start (s)', 'End (s)', 'Scientific name', 'Common name', 'Confidence'])
        writer.writerows(mock_data)
    return time.perf_counter() - start

def write_csv_pandas(path):
    start = time.perf_counter()
    df = pd.DataFrame(mock_data, columns=['Start (s)', 'End (s)', 'Scientific name', 'Common name', 'Confidence'])
    df.to_csv(path, index=False, sep=';')
    return time.perf_counter() - start

def write_csv_numpy(path):
    start = time.perf_counter()
    arr = np.array(mock_data, dtype=object)
    np.savetxt(path, arr, fmt='%s', delimiter=';', header='Start (s);End (s);Scientific name;Common name;Confidence', comments='')
    return time.perf_counter() - start


In [ ]:
print("==== READ COMPARISON (CPU) ====")
try:
    _, _, t1 = read_librosa(wav_path)
    print(f"Librosa:    {t1:.4f} s")
except Exception as e:
    print(f"Librosa failed: {e}")

try:
    _, _, t2 = read_scipy(wav_path)
    print(f"SciPy:      {t2:.4f} s")
except Exception as e:
    print(f"SciPy failed: {e}")

try:
    _, _, t3 = read_soundfile(wav_path)
    print(f"SoundFile:  {t3:.4f} s")
except Exception as e:
    print(f"SoundFile failed: {e}")

print("\n==== WRITE COMPARISON (CPU) ====")
t4 = write_csv_standard('out_standard.csv')
print(f"CSV module: {t4:.4f} s")

t5 = write_csv_pandas('out_pandas.csv')
print(f"Pandas:     {t5:.4f} s")

t6 = write_csv_numpy('out_numpy.csv')
print(f"Numpy:      {t6:.4f} s")

times = {
    "Librosa": t1 if 't1' in locals() else float('inf'),
    "SciPy": t2 if 't2' in locals() else float('inf'),
    "SoundFile": t3 if 't3' in locals() else float('inf'),
    "CSV": t4,
    "Pandas": t5,
    "Numpy": t6
}

print("\n==== SUMMARY (CPU) ====")
best_read = min(["Librosa", "SciPy", "SoundFile"], key=lambda k: times[k])
best_write = min(["CSV", "Pandas", "Numpy"], key=lambda k: times[k])
print(f"Best Read Method:  {best_read} ({times[best_read]:.4f} s)")
print(f"Best Write Method: {best_write} ({times[best_write]:.4f} s)")

==== READ COMPARISON (CPU) ====
Librosa:    0.1391 s
SciPy failed: cannot reshape array of size 5242481 into shape (2)
SoundFile:  0.0264 s

==== WRITE COMPARISON (CPU) ====
CSV module: 0.0003 s
Pandas:     0.0016 s
Numpy:      0.0004 s

==== SUMMARY (CPU) ====
Best Read Method:  SoundFile (0.0264 s)
Best Write Method: CSV (0.0003 s)


In [ ]:
print("==== READ COMPARISON (GPU) ====")
try:
    _, _, t1 = read_librosa(wav_path)
    print(f"Librosa:    {t1:.4f} s")
except Exception as e:
    print(f"Librosa failed: {e}")

try:
    _, _, t2 = read_scipy(wav_path)
    print(f"SciPy:      {t2:.4f} s")
except Exception as e:
    print(f"SciPy failed: {e}")

try:
    _, _, t3 = read_soundfile(wav_path)
    print(f"SoundFile:  {t3:.4f} s")
except Exception as e:
    print(f"SoundFile failed: {e}")

print("\n==== WRITE COMPARISON (GPU) ====")
t4 = write_csv_standard('out_standard.csv')
print(f"CSV module: {t4:.4f} s")

t5 = write_csv_pandas('out_pandas.csv')
print(f"Pandas:     {t5:.4f} s")

t6 = write_csv_numpy('out_numpy.csv')
print(f"Numpy:      {t6:.4f} s")

times = {
    "Librosa": t1 if 't1' in locals() else float('inf'),
    "SciPy": t2 if 't2' in locals() else float('inf'),
    "SoundFile": t3 if 't3' in locals() else float('inf'),
    "CSV": t4,
    "Pandas": t5,
    "Numpy": t6
}

print("\n==== SUMMARY (GPU) ====")
best_read = min(["Librosa", "SciPy", "SoundFile"], key=lambda k: times[k])
best_write = min(["CSV", "Pandas", "Numpy"], key=lambda k: times[k])
print(f"Best Read Method:  {best_read} ({times[best_read]:.4f} s)")
print(f"Best Write Method: {best_write} ({times[best_write]:.4f} s)")

==== READ COMPARISON (GPU) ====
Librosa:    0.1417 s
SciPy failed: cannot reshape array of size 5242481 into shape (2)
SoundFile:  0.0267 s

==== WRITE COMPARISON (GPU) ====
CSV module: 0.0002 s
Pandas:     0.0018 s
Numpy:      0.0004 s

==== SUMMARY (GPU) ====
Best Read Method:  SoundFile (0.0267 s)
Best Write Method: CSV (0.0002 s)


In [ ]:
print("==== READ COMPARISON (TPU) ====")
try:
    _, _, t1 = read_librosa(wav_path)
    print(f"Librosa:    {t1:.4f} s")
except Exception as e:
    print(f"Librosa failed: {e}")

try:
    _, _, t2 = read_scipy(wav_path)
    print(f"SciPy:      {t2:.4f} s")
except Exception as e:
    print(f"SciPy failed: {e}")

try:
    _, _, t3 = read_soundfile(wav_path)
    print(f"SoundFile:  {t3:.4f} s")
except Exception as e:
    print(f"SoundFile failed: {e}")

print("\n==== WRITE COMPARISON (TPU) ====")
t4 = write_csv_standard('out_standard.csv')
print(f"CSV module: {t4:.4f} s")

t5 = write_csv_pandas('out_pandas.csv')
print(f"Pandas:     {t5:.4f} s")

t6 = write_csv_numpy('out_numpy.csv')
print(f"Numpy:      {t6:.4f} s")

times = {
    "Librosa": t1 if 't1' in locals() else float('inf'),
    "SciPy": t2 if 't2' in locals() else float('inf'),
    "SoundFile": t3 if 't3' in locals() else float('inf'),
    "CSV": t4,
    "Pandas": t5,
    "Numpy": t6
}

print("\n==== SUMMARY (TPU) ====")
best_read = min(["Librosa", "SciPy", "SoundFile"], key=lambda k: times[k])
best_write = min(["CSV", "Pandas", "Numpy"], key=lambda k: times[k])
print(f"Best Read Method:  {best_read} ({times[best_read]:.4f} s)")
print(f"Best Write Method: {best_write} ({times[best_write]:.4f} s)")

==== READ COMPARISON (TPU) ====
Librosa:    0.1384 s
SciPy failed: cannot reshape array of size 5242481 into shape (2)
SoundFile:  0.0268 s

==== WRITE COMPARISON (TPU) ====
CSV module: 0.0002 s
Pandas:     0.0015 s
Numpy:      0.0003 s

==== SUMMARY (TPU) ====
Best Read Method:  SoundFile (0.0268 s)
Best Write Method: CSV (0.0002 s)


In [ ]:
# -------- OPTIONAL CLEANUP --------
import os
os.remove("out_standard.csv")
os.remove("out_pandas.csv")
os.remove("out_numpy.csv")